# Spatial Networks

## 📖 Ahead of time...

## 💻 Hands-on coding

### Streets as spatial graphs

### Summarising graphs

- Node centrality

### Attaching information to street segments

- Snap Airbnb's to street and include the network measures in their table as added columns

## 🐾 Next steps

If you found the content in this block useful, the following resources represent some suggestions on where to go next:

- Parts of the block benefit from the section on [urban networks](https://github.com/gboeing/ppd599/blob/master/modules/07-urban-networks-i/lecture.ipynb) in Geoff Boeing's excellent [course on Urban Data Science](https://github.com/gboeing/ppd599)
- If you are interested in urban morphometric analysis (the study of the shape of different elements making up cities), the [`momepy`](http://docs.momepy.org/en/stable/) library is an excellent reference to absorb, including its [user guide](http://docs.momepy.org/en/stable/user_guide/intro.html)
